# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Заказчик — кредитный отдел банка. Разберемся, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования могут быть учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***Импортируем библиотеки***

In [1]:
import pandas as pd

In [2]:
try:
    data= pd.read_csv( r'D:\Проекты яндекс-практикума\data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')  # Серверный путь

In [3]:
#data = pd.read_csv('/datasets/data.csv')
#data

### Шаг 2.1 Заполнение пропусков

In [4]:
data.info() 
#Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства (у нас именно такой случай)-поэтому заполняем пропуски медианным значением 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.isnull().sum()#пропуски обнаружились в столбцах days_employed и total_income. Могли возникнуть от того, что человек работает неофициально - нет подтвержденного трудового стажа и ежемесячного дохода

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
mediana_days_employed = data['days_employed'].median()#вычисляем медиану в столбце days_employed
abs(mediana_days_employed)

1203.369528770489

In [7]:
data['days_employed'].value_counts()

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

In [8]:
mediana_total_income = data['total_income'].median()#вычисляем медиану в столбце total_income
mediana_total_income

145017.93753253992

In [9]:
data['days_employed'] = data['days_employed'].fillna(mediana_days_employed)


In [10]:
data['total_income'] = data['total_income'].fillna(mediana_total_income)


In [11]:
data.isnull().sum()#проверяем, заполнились ли пропущенные значения. 


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Почему использовали медиану:Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства (у нас именно такой случай)-поэтому заполняем пропуски медианным значением 

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [12]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [13]:
data['education'] = data['education'].str.lower()#убираем заглавные буквы из столбца education


In [14]:
data['gender'] = data['gender'].str.lower()#убираем заглавные буквы из столбца gender

In [15]:
data = data.drop(columns=['days_employed'])#удаляем столбец days_employed	так как больше он нигде не используется
data.head(5) 

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


In [16]:
data['children'].unique()#смотрим уникальные значения в столбце

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [17]:
data['children']=data['children'].abs()#избавляемся от -1 ребенка, скорее всего это опечатка))


In [18]:
data['gender'].unique()#смотрим уникальные значения в столбце

array(['f', 'm', 'xna'], dtype=object)

20 детей- тоже сомнение. Но это могут быть усыновленные дети (такое бывает), поэтому решила оставить это значение

### Шаг 2.3. Изменение типов данных.

In [19]:
data['total_income'] = data['total_income'].astype('int')#заменяем total_income на int
data.info()#проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int64 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int32 
 10  purpose           21525 non-null  object
dtypes: int32(1), int64(5), object(5)
memory usage: 1.7+ MB


### Шаг 2.4. Удаление дубликатов.

In [20]:
data['purpose'].value_counts()#проверяем дубликаты в столбце purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [21]:
data['education'].value_counts()#проверяем дубликаты в столбце education - выясняем что их нет

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [22]:
data['family_status'].value_counts()#проверяем дубликаты в столбце family_status- выясняем что их нет

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [23]:
data['income_type'].value_counts()#проверяем дубликаты в столбце income_type- выясняем что их нет

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [25]:
education_log = data[['education','education_id']] # создаем "словари" для столбцов образования
education_log.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
5,высшее,0
6,высшее,0
7,среднее,1
8,высшее,0
9,среднее,1


In [26]:
family_status_log = data[['family_status','family_status_id']] # создаем "словари" для столбцов семейного положения
print(family_status_log.head(10))

      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


In [27]:
education_log = education_log.drop_duplicates().reset_index(drop=True)
print(education_log.head()) #удаляем дубликаты в "словаре"

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [28]:
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
print(family_status_log.head()) #удаляем дубликаты в "словаре"

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


In [29]:
data = data.drop(['education', 'family_status'], axis = 1)
data.head(5)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,53,1,1,f,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [30]:
def total_income_category(total_income):# создадим функцию для категоризации по доходу ('E', если 0–30000;'D', если 30001–50000;'C', если 50001–200000'B', если 200001–1000000,'A', если 1000001 и выше)
    if  total_income <=30000:
        return 'E'
    elif 30001<= total_income <=50000:
        return 'D'
    elif 50001<= total_income <=200000:
        return 'C'
    elif 200001<= total_income <=1000000:
        return 'B'
    else:   
        return('A')


In [31]:
data['total_income_category'] = data['total_income'].apply(total_income_category)

data.head(10)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,42,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,33,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,32,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C
5,0,27,0,1,m,компаньон,0,255763,покупка жилья,B
6,0,43,0,0,f,компаньон,0,240525,операции с жильем,B
7,0,50,1,0,m,сотрудник,0,135823,образование,C
8,2,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C
9,0,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [32]:
def purpose_category(purpose):
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    return 'операции с недвижимостью'

#purpose_category('сделка с автомобилем') #проверяем, работает ли функция

In [33]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

data.head(10)

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,42,0,0,f,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,33,1,0,m,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,32,1,0,m,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,27,0,1,m,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,43,0,0,f,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,50,1,0,m,сотрудник,0,135823,образование,C,получение образования
8,2,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1 (про детей): Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [34]:
data_pivot_1 = data.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')#посмотрим,сколько людей с различным кол-вом детей имеют задолженность по кредитам
data_pivot_1

debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1861.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


In [35]:
data_pivot_1['new'] = data_pivot_1[0] + data_pivot_1 [1] #создаем доп столбец суммарный
data_pivot_1

debt,0,1,new
children,,,
0,13086.0,1063.0,14149.0
1,4420.0,445.0,4865.0
2,1861.0,194.0,2055.0
3,303.0,27.0,330.0
4,37.0,4.0,41.0
5,9.0,NaN,NaN
20,68.0,8.0,76.0


In [36]:
data_pivot_1['new_1'] = data_pivot_1[1]/data_pivot_1['new'] # ищем в долях,сколько не выплачивают в срок относительно всех,имеющих одинаковое кол-во детей
data_pivot_1

debt,0,1,new,new_1
children,,,,
0,13086.0,1063.0,14149.0,0.075129
1,4420.0,445.0,4865.0,0.091470
2,1861.0,194.0,2055.0,0.094404
3,303.0,27.0,330.0,0.081818
4,37.0,4.0,41.0,0.097561
5,9.0,NaN,NaN,NaN
20,68.0,8.0,76.0,0.105263


По данным можно судить, что доля невыплаченных кредитов в срок (столбец new_1) незначительно увеличивается с ростом кол-ва детей в семье. Соответственно, делаем вывод, что кол-во детей влияет на возврат кредита в срок - чем их больше, тем больше вероятность его не выплатить

##### Вопрос 2 (про семейное положение): Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
data_pivot_1 = data.pivot_table(index=['family_status_id'], columns='debt', values='children', aggfunc='count')#посмотрим,сколько людей с различным кол-вом детей имеют задолженность по кредитам
data_pivot_1

debt,0,1
family_status_id,,
0,11449,931
1,3789,388
2,897,63
3,1110,85
4,2539,274


In [38]:
data_pivot_1['new'] = data_pivot_1[0] + data_pivot_1 [1]
data_pivot_1

debt,0,1,new
family_status_id,,,
0,11449,931,12380
1,3789,388,4177
2,897,63,960
3,1110,85,1195
4,2539,274,2813


In [39]:
data_pivot_1['new_1'] = data_pivot_1[1]/data_pivot_1['new'] # ищем в долях,сколько не выплачивают в срок относительно всех,имеющих один пол
data_pivot_1

debt,0,1,new,new_1
family_status_id,,,,
0,11449,931,12380,0.075202
1,3789,388,4177,0.092890
2,897,63,960,0.065625
3,1110,85,1195,0.071130
4,2539,274,2813,0.097405


По данным можно судить, что доля невыплаченных кредитов в срок (столбец new_1)  больше у людей, не состоящих в браке или состоящих в так называемом гражданском браке. Соответственно, делаем вывод, что семейное положение влияет на возврат кредита в срок - свободные люди чувствуют себя свободными об обязательств, в том числе и долговых)))

##### Вопрос 3 (про доход): Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
data_pivot_1 = data.pivot_table(index=['total_income_category'], columns='debt', values='gender', aggfunc='count')#посмотрим,сколько людей с различным кол-вом дохода имеют задолженность по кредитам
data_pivot_1


debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14727,1360
D,329,21
E,20,2


In [41]:
data_pivot_1['new'] = data_pivot_1[0] + data_pivot_1 [1]
data_pivot_1

debt,0,1,new
total_income_category,,,
A,23,2,25
B,4685,356,5041
C,14727,1360,16087
D,329,21,350
E,20,2,22


In [42]:
data_pivot_1['new_1'] = data_pivot_1[1]/data_pivot_1['new'] # ищем в долях,сколько не выплачивают в срок относительно всех
data_pivot_1

debt,0,1,new,new_1
total_income_category,,,,
A,23,2,25,0.080000
B,4685,356,5041,0.070621
C,14727,1360,16087,0.084540
D,329,21,350,0.060000
E,20,2,22,0.090909


По данным можно судить о том, что доход людей влияет на выплату кредита в срок, доли людей с доходом 0–30000 имеют задолженности гораздо чаще. Это и понятно- это либо безработные, либо студенты или пенсионеры. Также большой процент невыплаченных в срок кредитов у людей с большими доходами -1000001 и выше. Они мыслят большими деньгами и оплата кредита для них незначительна, они часто пропускают платеж и платят пени, для них они необременительны. Понятно, люди озабочены зарабатыванием миллиардов, а не выплачиваем долгов))

##### Вопрос 4(про кредитные цели): Как разные цели кредита влияют на его возврат в срок?

In [43]:
data_pivot_1 = data.pivot_table(index=['purpose_category'], columns='debt', values='gender', aggfunc='count')#посмотрим,сколько людей с различным кол-вом дохода имеют задолженность по кредитам
data_pivot_1

debt,0,1
purpose_category,,
операции с автомобилем,3912,403
операции с недвижимостью,10058,782
получение образования,3652,370
проведение свадьбы,2162,186


In [44]:
data_pivot_1['new'] = data_pivot_1[0] + data_pivot_1 [1]
data_pivot_1

debt,0,1,new
purpose_category,,,
операции с автомобилем,3912,403,4315
операции с недвижимостью,10058,782,10840
получение образования,3652,370,4022
проведение свадьбы,2162,186,2348


In [45]:
data_pivot_1['new_1'] = data_pivot_1[1]/data_pivot_1['new'] # ищем в долях,сколько не выплачивают в срок относительно всех
data_pivot_1

debt,0,1,new,new_1
purpose_category,,,,
операции с автомобилем,3912,403,4315,0.093395
операции с недвижимостью,10058,782,10840,0.072140
получение образования,3652,370,4022,0.091994
проведение свадьбы,2162,186,2348,0.079216


Можно сказать, что незначительное превышение в долях невыплативших кредитв срок у людей, цель которых провести операции с автомобилем и получить образование. Возможно это связано с тем, что эти люди более молодые 

## Общий вывод:

После проделанного исследования можно сказать, что есть различные критерии, по которым можно судить о том, выплатит ли заемщик кредит в срок.
1) Наличие детей: чем больше детей, тем меньшая вероятность, что кредит будет выплачен.  
2) Семейное положение: люди, не связанные узами брака- сомнительные заемщики. Лучше дадим кредит семьянинам!    
3) Доход: хуже всего выплачивают кредит либо люди с отсутствием или крайне низким доходом, либо очень обеспеченные люди. Даешь кредит среднему классу!))  
4) Цели: людям, кредитные цели которых получение образования и операции с автомобилем - лучше кредит не давать, весьма вероятно, что они не смогут выплатить его в срок.  